In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
import talib  # Technical Analysis library for ADX calculation
import logging
import time

logging.basicConfig(level=logging.INFO)

nse_tickers = [
    "DMART.NS","TCS.NS", "BEL.NS", "DLF.NS", "GODREJCP.NS", "HDFCLIFE.NS",
    "MAXHEALTH.NS", "SBILIFE.NS", "TRENT.NS", "VBL.NS", "KPIL.NS", "GMRPUI.NS",
    "SWIGGY.NS","ETERNAL.NS","PAYTM.NS","ADANIPOWER.NS","HEG.NS","PAYTM.NS"
]

def safe_scalar(val):
    try:
        if isinstance(val, pd.Series):
            if not val.empty:
                return float(val.iloc[0])
            else:
                return np.nan
        else:
            return float(val)
    except:
        return np.nan

def fetch_yfinance_data(ticker):
    return yf.download(ticker, period="90d", interval="1d", auto_adjust=True)

def calculate_rsi(df, window=14):
    delta = df['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -1 * delta.clip(upper=0)
    avg_gain = gain.rolling(window=window, min_periods=window).mean()
    avg_loss = loss.rolling(window=window, min_periods=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_ma(df, windows=[10, 20, 50]):
    for w in windows:
        df[f'MA_{w}'] = df['Close'].rolling(window=w).mean()
    return df

def calculate_adx(df, window=14):
    # Ensure no MultiIndex columns
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    high = df['High'].dropna().values
    low = df['Low'].dropna().values
    close = df['Close'].dropna().values

    min_len = min(len(high), len(low), len(close))
    high = high[-min_len:]
    low = low[-min_len:]
    close = close[-min_len:]

    adx_vals = talib.ADX(high, low, close, timeperiod=window)

    # Align ADX length with original df by prepending NaNs
    adx_full = np.full(shape=len(df), fill_value=np.nan)
    adx_full[-len(adx_vals):] = adx_vals

    return adx_full

def find_swing_zones(df, order=5):
    local_min_idx = argrelextrema(df['Close'].values, np.less_equal, order=order)[0]
    demand_zones = df.iloc[local_min_idx]

    local_max_idx = argrelextrema(df['Close'].values, np.greater_equal, order=order)[0]
    supply_zones = df.iloc[local_max_idx]

    return demand_zones, supply_zones

def calc_support_resistance_zone(df, window=10, pct_range=0.02):
    recent_data = df.tail(window)
    low = safe_scalar(recent_data['Low'].min())
    high = safe_scalar(recent_data['High'].max())
    support_zone_low = low
    support_zone_high = low * (1 + pct_range) if pd.notna(low) else np.nan
    resistance_zone_low = high * (1 - pct_range) if pd.notna(high) else np.nan
    resistance_zone_high = high
    return support_zone_low, support_zone_high, resistance_zone_low, resistance_zone_high

def generate_score(rsi, ma_10, ma_20, ma_50, volume, volavg20, adx, adx_threshold=20):
    score = 0
    reasons = []

    if float(rsi) < 30:
        score += 2
        reasons.append('RSI oversold')
    elif float(rsi) > 70:
        score -= 2
        reasons.append('RSI overbought')
    elif 45 <= float(rsi) <= 55:
        score += 1
        reasons.append('RSI neutral strong')

    if float(ma_10) > float(ma_20) and float(ma_20) > float(ma_50):
        score += 2
        reasons.append('MA 10 > MA 20 > MA 50 strong uptrend')
    elif float(ma_10) < float(ma_20) and float(ma_20) < float(ma_50):
        score -= 2
        reasons.append('MA 10 < MA 20 < MA 50 strong downtrend')

    if float(volume) > 1.5 * float(volavg20):
        score += 1
        reasons.append('High volume')

    if adx is not None and not np.isnan(adx):
        if adx >= adx_threshold:
            score += 1
            reasons.append(f'ADX strong trend >= {adx_threshold}')
        else:
            score -= 1
            reasons.append(f'ADX weak trend < {adx_threshold}')

    if score >= 3:
        signal = 'BUY'
    elif score <= -2:
        signal = 'SELL'
    else:
        signal = 'HOLD'

    if not reasons:
        reasons.append('No strong signals')

    return score, signal, '; '.join(reasons)

def calc_trade_levels(price, demand_zone, supply_zone, support_low, support_high, resistance_low, resistance_high, signal):
    if signal != 'BUY':
        return None, None, None, None, None

    entry = price if price > demand_zone else demand_zone if demand_zone else price
    stop_loss = demand_zone * 0.98 if demand_zone else price * 0.98

    targets = []
    if resistance_low and resistance_low > entry:
        targets.append(resistance_low)
    if resistance_high and resistance_high > entry:
        targets.append(resistance_high)
    if supply_zone and supply_zone > entry:
        targets.append(supply_zone)

    while len(targets) < 3:
        targets.append(None)

    return round(entry, 2), round(stop_loss, 2), round(targets[0], 2) if targets[0] else None, round(targets[1], 2) if targets[1] else None, round(targets[2], 2) if targets[2] else None


results = []

for ticker in nse_tickers:
    logging.info(f'Processing {ticker}...')
    df = fetch_yfinance_data(ticker)
    if df.empty:
        logging.warning(f'No data for {ticker}, skipping.')
        continue

    df = calculate_ma(df)
    df['RSI'] = calculate_rsi(df)
    df['VolAvg20'] = df['Volume'].rolling(20).mean()
    df['ADX'] = calculate_adx(df)

    demand_zones, supply_zones = find_swing_zones(df, order=5)

    latest = df.iloc[-1]
    last_date = latest.name

    demand_recent = demand_zones[demand_zones.index < last_date]
    demand_zone_price = safe_scalar(demand_recent['Close'].iloc[-1]) if not demand_recent.empty else np.nan
    demand_zone_val = round(float(demand_zone_price), 2) if pd.notna(demand_zone_price) else None

    supply_recent = supply_zones[supply_zones.index < last_date]
    supply_zone_price = safe_scalar(supply_recent['Close'].iloc[-1]) if not supply_recent.empty else np.nan
    supply_zone_val = round(float(supply_zone_price), 2) if pd.notna(supply_zone_price) else None

    support_zone_low, support_zone_high, resistance_zone_low, resistance_zone_high = calc_support_resistance_zone(df)

    price_val = safe_scalar(latest['Close'])
    rsi_val = safe_scalar(latest['RSI'])
    ma_10_val = safe_scalar(latest['MA_10'])
    ma_20_val = safe_scalar(latest['MA_20'])
    ma_50_val = safe_scalar(latest['MA_50'])
    volume_val = safe_scalar(latest['Volume'])
    volavg20_val = safe_scalar(latest['VolAvg20'])
    adx_val = safe_scalar(latest['ADX'])

    score, signal, reasons = generate_score(rsi_val, ma_10_val, ma_20_val, ma_50_val, volume_val, volavg20_val, adx_val)

    entry, stop_loss, target1, target2, target3 = calc_trade_levels(
        price_val, demand_zone_val, supply_zone_val,
        support_zone_low, support_zone_high,
        resistance_zone_low, resistance_zone_high,
        signal
    )

    result = {
        'Ticker': ticker.replace('.NS',''),
        'Price': round(price_val, 2) if pd.notna(price_val) else None,
        'RSI': round(rsi_val, 2) if pd.notna(rsi_val) else None,
        'MA_10': round(ma_10_val, 2) if pd.notna(ma_10_val) else None,
        'MA_20': round(ma_20_val, 2) if pd.notna(ma_20_val) else None,
        'MA_50': round(ma_50_val, 2) if pd.notna(ma_50_val) else None,
        'Volume': int(volume_val) if pd.notna(volume_val) else None,
        'VolAvg20': int(volavg20_val) if pd.notna(volavg20_val) else None,
        'ADX': round(adx_val, 2) if pd.notna(adx_val) else None,
        'SupportZoneLow': round(support_zone_low, 2) if pd.notna(support_zone_low) else None,
        'SupportZoneHigh': round(support_zone_high, 2) if pd.notna(support_zone_high) else None,
        'ResistanceZoneLow': round(resistance_zone_low, 2) if pd.notna(resistance_zone_low) else None,
        'ResistanceZoneHigh': round(resistance_zone_high, 2) if pd.notna(resistance_zone_high) else None,
        'DemandZone': demand_zone_val,
        'SupplyZone': supply_zone_val,
        'Entry': entry,
        'StopLoss': stop_loss,
        'Target1': target1,
        'Target2': target2,
        'Target3': target3,
        'Score': score,
        'Signal': signal,
        'Reasons': reasons
    }

    results.append(result)
    time.sleep(1)  # Respect API limits

df_out = pd.DataFrame(results)
print(df_out)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: GMRPUI.NS"}}}
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed 

        Ticker    Price    RSI    MA_10    MA_20    MA_50     Volume  \
0        DMART  4576.20  34.57  4674.38  4699.52  4467.31     454331   
1          TCS  2957.40  39.88  3103.87  3094.58  3090.40    4971744   
2          BEL   403.15  72.05   404.11   390.00   386.58   23800929   
3          DLF   721.05  34.28   765.60   759.77   774.74    4484651   
4     GODREJCP  1189.20  37.31  1225.70  1239.50  1232.31    1908827   
5     HDFCLIFE   765.05  52.83   776.42   772.89   768.89    2080889   
6    MAXHEALTH  1137.70  40.84  1160.47  1162.83  1209.25    5021970   
7      SBILIFE  1809.80  50.94  1824.27  1815.70  1826.60    1003675   
8        TRENT  4742.50   7.88  5037.30  5198.38  5259.96    1517967   
9          VBL   451.85  34.36   465.03   474.77   491.72    8205942   
10        KPIL  1255.10  46.90  1269.56  1264.65  1225.97     110720   
11      SWIGGY   426.10  43.32   438.98   433.71   417.62    8852879   
12     ETERNAL   332.25  53.18   332.29   328.48   313.75   1665

In [2]:
pip install TA-lib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 33.9 MB/s eta 0:00:00


In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
import talib
import logging
import time

logging.basicConfig(level=logging.INFO)

nse_tickers = [
    "DMART.NS","TCS.NS", "BEL.NS", "DLF.NS", "GODREJCP.NS", "HDFCLIFE.NS",
    "MAXHEALTH.NS", "SBILIFE.NS", "TRENT.NS", "VBL.NS", "KPIL.NS", "GMRPUI.NS",
    "SWIGGY.NS","ETERNAL.NS","PAYTM.NS","ADANIPOWER.NS","HEG.NS","PAYTM.NS"
]

INITIAL_CAPITAL = 10000  # Starting capital per ticker
RISK_PERCENTAGE = 0.01  # Risk 1% per trade
ATR_MULTIPLIER_STOP = 1.5  # Stop loss multiplier for ATR
ATR_MULTIPLIER_TARGET = 3  # Profit target multiplier for ATR

def safe_scalar(val):
    try:
        if isinstance(val, pd.Series):
            if not val.empty:
                return float(val.iloc[0])
            else:
                return np.nan
        else:
            return float(val)
    except:
        return np.nan

def fetch_yfinance_data(ticker):
    return yf.download(ticker, period="90d", interval="1d", auto_adjust=True)

def calculate_rsi(df, window=14):
    delta = df['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -1 * delta.clip(upper=0)
    avg_gain = gain.rolling(window=window, min_periods=window).mean()
    avg_loss = loss.rolling(window=window, min_periods=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_ma(df, windows=[10, 20, 50]):
    for w in windows:
        df[f'MA_{w}'] = df['Close'].rolling(window=w).mean()
    return df

def calculate_adx(df, window=14):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    high = df['High'].dropna().values
    low = df['Low'].dropna().values
    close = df['Close'].dropna().values

    min_len = min(len(high), len(low), len(close))
    high = high[-min_len:]
    low = low[-min_len:]
    close = close[-min_len:]

    adx_vals = talib.ADX(high, low, close, timeperiod=window)

    adx_full = np.full(shape=len(df), fill_value=np.nan)
    adx_full[-len(adx_vals):] = adx_vals

    return adx_full

def calculate_atr(df, window=14):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    high = df['High'].values
    low = df['Low'].values
    close = df['Close'].values
    atr_vals = talib.ATR(high, low, close, timeperiod=window)
    atr_full = np.full(shape=len(df), fill_value=np.nan)
    atr_full[-len(atr_vals):] = atr_vals
    return atr_full

def find_swing_zones(df, order=5):
    local_min_idx = argrelextrema(df['Close'].values, np.less_equal, order=order)[0]
    demand_zones = df.iloc[local_min_idx]

    local_max_idx = argrelextrema(df['Close'].values, np.greater_equal, order=order)[0]
    supply_zones = df.iloc[local_max_idx]

    return demand_zones, supply_zones

def calc_support_resistance_zone(df, window=10, pct_range=0.02):
    recent_data = df.tail(window)
    low = safe_scalar(recent_data['Low'].min())
    high = safe_scalar(recent_data['High'].max())
    support_zone_low = low
    support_zone_high = low * (1 + pct_range) if pd.notna(low) else np.nan
    resistance_zone_low = high * (1 - pct_range) if pd.notna(high) else np.nan
    resistance_zone_high = high
    return support_zone_low, support_zone_high, resistance_zone_low, resistance_zone_high

def generate_score(rsi, ma_10, ma_20, ma_50, volume, volavg20, adx, adx_threshold=20):
    score = 0
    reasons = []

    if float(rsi) < 30:
        score += 2
        reasons.append('RSI oversold')
    elif float(rsi) > 70:
        score -= 2
        reasons.append('RSI overbought')
    elif 45 <= float(rsi) <= 55:
        score += 1
        reasons.append('RSI neutral strong')

    if float(ma_10) > float(ma_20) and float(ma_20) > float(ma_50):
        score += 2
        reasons.append('MA 10 > MA 20 > MA 50 strong uptrend')
    elif float(ma_10) < float(ma_20) and float(ma_20) < float(ma_50):
        score -= 2
        reasons.append('MA 10 < MA 20 < MA 50 strong downtrend')

    if float(volume) > 1.5 * float(volavg20):
        score += 1
        reasons.append('High volume')

    if adx is not None and not np.isnan(adx):
        if adx >= adx_threshold:
            score += 1
            reasons.append(f'ADX strong trend >= {adx_threshold}')
        else:
            score -= 1
            reasons.append(f'ADX weak trend < {adx_threshold}')

    if score >= 3:
        signal = 'BUY'
    elif score <= -2:
        signal = 'SELL'
    else:
        signal = 'HOLD'

    if not reasons:
        reasons.append('No strong signals')

    return score, signal, '; '.join(reasons)

def calc_trade_levels(price, atr, signal):
    if signal != 'BUY':
        return None, None, None

    entry = price
    stop_loss = price - ATR_MULTIPLIER_STOP * atr if atr and not np.isnan(atr) else price * 0.98
    target = price + ATR_MULTIPLIER_TARGET * atr if atr and not np.isnan(atr) else price * 1.05

    return round(entry, 2), round(stop_loss, 2), round(target, 2)

def calculate_position_size(capital, risk_per_trade, entry_price, stop_loss):
    if stop_loss is None or entry_price is None:
        return 0
    risk_amount = capital * risk_per_trade
    risk_per_share = abs(entry_price - stop_loss)
    if risk_per_share == 0:
        return 0
    size = risk_amount / risk_per_share
    return int(size)

results = []

for ticker in nse_tickers:
    logging.info(f'Processing {ticker}...')
    df = fetch_yfinance_data(ticker)
    if df.empty:
        logging.warning(f'No data for {ticker}, skipping.')
        continue

    df = calculate_ma(df)
    df['RSI'] = calculate_rsi(df)
    df['VolAvg20'] = df['Volume'].rolling(20).mean()
    df['ADX'] = calculate_adx(df)
    df['ATR'] = calculate_atr(df)

    latest = df.iloc[-1]

    price = safe_scalar(latest['Close'])
    rsi = safe_scalar(latest['RSI'])
    ma_10 = safe_scalar(latest['MA_10'])
    ma_20 = safe_scalar(latest['MA_20'])
    ma_50 = safe_scalar(latest['MA_50'])
    volume = safe_scalar(latest['Volume'])
    volavg20 = safe_scalar(latest['VolAvg20'])
    adx = safe_scalar(latest['ADX'])
    atr = safe_scalar(latest['ATR'])

    score, signal, reasons = generate_score(rsi, ma_10, ma_20, ma_50, volume, volavg20, adx)

    entry, stop_loss, target = calc_trade_levels(price, atr, signal)
    position_size = calculate_position_size(INITIAL_CAPITAL, RISK_PERCENTAGE, entry, stop_loss)

    result = {
        'Ticker': ticker.replace('.NS',''),
        'Price': round(price, 2) if price else None,
        'RSI': round(rsi, 2) if rsi else None,
        'MA_10': round(ma_10, 2) if ma_10 else None,
        'MA_20': round(ma_20, 2) if ma_20 else None,
        'MA_50': round(ma_50, 2) if ma_50 else None,
        'Volume': int(volume) if volume else None,
        'VolAvg20': int(volavg20) if volavg20 else None,
        'ADX': round(adx, 2) if adx else None,
        'ATR': round(atr, 2) if atr else None,
        'Score': score,
        'Signal': signal,
        'Entry': entry,
        'StopLoss': stop_loss,
        'Target': target,
        'PositionSize': position_size,
        'Reasons': reasons
    }

    results.append(result)
    time.sleep(1)  # API rate limit respect

df_out = pd.DataFrame(results)
print(df_out)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: GMRPUI.NS"}}}
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed 

        Ticker    Price    RSI    MA_10    MA_20    MA_50     Volume  \
0        DMART  4576.20  34.57  4674.38  4699.52  4467.31     454331   
1          TCS  2957.40  39.88  3103.87  3094.58  3090.40    4971744   
2          BEL   403.15  72.05   404.11   390.00   386.58   23800929   
3          DLF   721.05  34.28   765.60   759.77   774.74    4484651   
4     GODREJCP  1189.20  37.31  1225.70  1239.50  1232.31    1908827   
5     HDFCLIFE   765.05  52.83   776.42   772.89   768.89    2080889   
6    MAXHEALTH  1137.70  40.84  1160.47  1162.83  1209.25    5021970   
7      SBILIFE  1809.80  50.94  1824.27  1815.70  1826.60    1003675   
8        TRENT  4742.50   7.88  5037.30  5198.38  5259.96    1517967   
9          VBL   451.85  34.36   465.03   474.77   491.72    8205942   
10        KPIL  1255.10  46.90  1269.56  1264.65  1225.97     110720   
11      SWIGGY   426.10  43.32   438.98   433.71   417.62    8852879   
12     ETERNAL   332.25  53.18   332.29   328.48   313.75   1665

In [18]:
import backtrader as bt
import yfinance as yf
import pandas as pd

class SwingStrategy(bt.Strategy):
    params = dict(
        rsi_period=14,
        ma_periods=(10, 20, 50),
        adx_period=14,
        atr_period=14,
        atr_mult_stop=1.5,
        atr_mult_target=3,
        risk_per_trade=0.01,
    )

    def __init__(self):
        self.rsi = bt.indicators.RSI(self.data.close, period=self.p.rsi_period)
        self.ma10 = bt.indicators.SimpleMovingAverage(self.data.close, period=self.p.ma_periods[0])
        self.ma20 = bt.indicators.SimpleMovingAverage(self.data.close, period=self.p.ma_periods[1])
        self.ma50 = bt.indicators.SimpleMovingAverage(self.data.close, period=self.p.ma_periods[2])
        self.adx = bt.indicators.ADX(self.data, period=self.p.adx_period)
        self.atr = bt.indicators.ATR(self.data, period=self.p.atr_period)

        self.order = None
        self.stop_price = None
        self.position_size = 0

    def next(self):
        if self.order:
            return  # waiting for pending order

        score = 0
        if self.rsi[0] < 30:
            score += 2
        elif self.rsi[0] > 70:
            score -= 2
        elif 45 <= self.rsi[0] <= 55:
            score += 1

        if self.ma10[0] > self.ma20[0] > self.ma50[0]:
            score += 2
        elif self.ma10[0] < self.ma20[0] < self.ma50[0]:
            score -= 2

        if self.adx[0] >= 20:
            score += 1
        else:
            score -= 1

        if not self.position:
            if score >= 3:
                stop_loss = self.data.close[0] - self.p.atr_mult_stop * self.atr[0]
                risk_per_share = self.data.close[0] - stop_loss
                cash = self.broker.getcash()
                size = int(cash * self.p.risk_per_trade / risk_per_share)
                if size > 0:
                    self.position_size = size
                    self.stop_price = stop_loss
                    self.order = self.buy(size=size)
        else:
            new_stop = self.data.close[0] - self.p.atr_mult_stop * self.atr[0]
            if new_stop > self.stop_price:
                self.stop_price = new_stop
            if self.data.close[0] <= self.stop_price:
                self.order = self.sell(size=self.position_size)
                self.position_size = 0
                self.stop_price = None


def run_backtest_for_ticker(ticker):
    print(f"Downloading data for {ticker} ...")
    df = yf.download(ticker, start='2023-01-01', end='2023-12-31', group_by='column', auto_adjust=True)

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    df.index = df.index.tz_localize(None)

    cerebro = bt.Cerebro()
    data_feed = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(data_feed)
    cerebro.broker.setcash(10000)
    cerebro.addstrategy(SwingStrategy)

    print(f"Starting Portfolio Value for {ticker}: {cerebro.broker.getvalue():.2f}")
    cerebro.run()
    print(f"Final Portfolio Value for {ticker}: {cerebro.broker.getvalue():.2f}")
    print("="*50)


nse_tickers = [
    "DMART.NS", "TCS.NS", "BEL.NS", "DLF.NS", "GODREJCP.NS", "HDFCLIFE.NS",
    "MAXHEALTH.NS", "SBILIFE.NS", "TRENT.NS", "VBL.NS", "KPIL.NS",
    "SWIGGY.NS", "ETERNAL.NS", "PAYTM.NS", "ADANIPOWER.NS", "HEG.NS"
]

for ticker in nse_tickers:
    try:
        run_backtest_for_ticker(ticker)
    except Exception as e:
        print(f"Error running backtest for {ticker}: {e}")
        print("="*50)


[*********************100%***********************]  1 of 1 completed

Starting Portfolio Value for DMART.NS: 10000.00



[*********************100%***********************]  1 of 1 completed


Final Portfolio Value for DMART.NS: 10622.55
Starting Portfolio Value for TCS.NS: 10000.00
Final Portfolio Value for TCS.NS: 10339.26


[*********************100%***********************]  1 of 1 completed


...

Final Portfolio Value for ETERNAL.NS: 11567.50
Starting Portfolio Value for PAYTM.NS: 10000.00



[*********************100%***********************]  1 of 1 completed

Final Portfolio Value for PAYTM.NS: 9952.90
Starting Portfolio Value for ADANIPOWER.NS: 10000.00



[*********************100%***********************]  1 of 1 completed

Final Portfolio Value for ADANIPOWER.NS: 12147.11
Starting Portfolio Value for HEG.NS: 10000.00
Final Portfolio Value for HEG.NS: 11891.16


In [21]:
import backtrader as bt
import yfinance as yf
import pandas as pd

# Your SwingStrategy definition here (omitted for brevity)

def run_backtest_for_ticker(ticker):
    df = yf.download(ticker, start='2023-01-01', end='2023-12-31', group_by='column', auto_adjust=True)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df.index = df.index.tz_localize(None)

    cerebro = bt.Cerebro()
    data_feed = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(data_feed)
    cerebro.broker.setcash(10000)
    cerebro.addstrategy(SwingStrategy)

    starting_value = cerebro.broker.getvalue()
    cerebro.run()
    final_value = cerebro.broker.getvalue()

    return {
        'Ticker': ticker,
        'Starting Value': starting_value,
        'Final Value': final_value,
        'Net Profit (%)': (final_value - starting_value) / starting_value * 100
    }

nse_tickers = [
    "DMART.NS", "TCS.NS", "BEL.NS", "DLF.NS", "GODREJCP.NS", "HDFCLIFE.NS",
    "MAXHEALTH.NS", "SBILIFE.NS", "TRENT.NS", "VBL.NS", "KPIL.NS",
    "ETERNAL.NS", "PAYTM.NS", "ADANIPOWER.NS", "HEG.NS","PAYTM.NS","SWIGGY.NS"
]

results = []
for ticker in nse_tickers:
    try:
        res = run_backtest_for_ticker(ticker)
        results.append(res)
        print(f"Backtest completed for {ticker}: {res['Net Profit (%)']:.2f}%")
    except Exception as e:
        print(f"Error running backtest for {ticker}: {e}")

df_results = pd.DataFrame(results)
print(df_results)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Backtest completed for DMART.NS: 6.23%
Backtest completed for TCS.NS: 3.39%



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Backtest completed for BEL.NS: 23.06%
Backtest completed for DLF.NS: 18.68%



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

...

Backtest completed for KPIL.NS: 0.13%
Backtest completed for ETERNAL.NS: 15.67%



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Backtest completed for PAYTM.NS: -0.47%
Backtest completed for ADANIPOWER.NS: 21.47%



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SWIGGY.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-01-01 -> 2023-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1672511400, endDate = 1703961000")')


Backtest completed for HEG.NS: 18.91%
Backtest completed for PAYTM.NS: -0.47%
Error running backtest for SWIGGY.NS: array index out of range
           Ticker  Starting Value   Final Value  Net Profit (%)
0        DMART.NS           10000  10622.549805        6.225498
1          TCS.NS           10000  10339.255645        3.392556
2          BEL.NS           10000  12306.433906       23.064339
3          DLF.NS           10000  11867.653932       18.676539
4     GODREJCP.NS           10000  10527.816586        5.278166
5     HDFCLIFE.NS           10000  10576.192381        5.761924
6    MAXHEALTH.NS           10000  11057.386223       10.573862
7      SBILIFE.NS           10000  10741.852547        7.418525
8        TRENT.NS           10000  13128.118764       31.281188
9          VBL.NS           10000  11472.930326       14.729303
10        KPIL.NS           10000  10013.304795        0.133048
11     ETERNAL.NS           10000  11567.499924       15.674999
12       PAYTM.NS          